In [1]:
# To setup everything (on the bash terminal to execute jupyter nb)
# !./install.sh
# !. ./env.sh
# !which python
# !jupyter notebook --ip="0.0.0.0" 

In [2]:
import pkwrap.infer_helper as demo
import importlib; importlib.reload(demo);

In [3]:
models = []

forward_asr_wav2vec2, pk_model = demo.init_asr_model(
    model="local/chain/e2e/tuning/tdnnf_wav2vec.py",
    exp_path="exp/chain/e2e_tdnnf_wav2vec/",
)

forward_asr_wav2vec2_vq_48, pk_model = demo.init_asr_model(
    model=f"local/chain/e2e/tuning/tdnnf_wav2vec_vq_lds.py",
    exp_path=f"exp/chain/e2e_tdnnf_wav2vec_lds_vq_{48}/",
    pkwrap_vq_dim=48,
)

forward_asr, pk_model = demo.init_asr_model(
    model="local/chain/e2e/tuning/tdnnf.py",
    exp_path="exp/chain/e2e_tdnnf/",
)
forward_synt, synt_model = demo.init_synt_model(
    model=f"local/tuning/hifi_gan.py",
    exp_path=f"exp/hifigan",
    asr_bn_model = pk_model,
    model_weight = "g_00102000",
)
models.append({"asr": forward_asr, "synthesis": forward_synt, "dim": "no_vq"})

vq_nb_proto = [16, 32, 48, 64, 128, 256, 512, 1024]
for dim in vq_nb_proto:

    forward_asr, pk_model = demo.init_asr_model(
        model=f"local/chain/e2e/tuning/tdnnf_vq_bd.py",
        exp_path=f"exp/chain/e2e_tdnnf_vq_{dim}/",
        pkwrap_vq_dim=dim,
    )
    if dim == 1024:
        forward_synt, synt_model = demo.init_synt_model(
            model=f"local/tuning/hifi_gan.py",
            exp_path=f"exp/hifigan_vq_{dim}_finetuned",
            asr_bn_model = pk_model,
        )

        models.append({f"asr": forward_asr, "synthesis": forward_synt, "dim": dim})
        continue
    forward_synt, synt_model = demo.init_synt_model(
        model=f"local/tuning/hifi_gan.py",
        exp_path=f"exp/hifigan_vq_{dim}_finetuned",
        asr_bn_model = pk_model,
        model_weight = "g_00075000",
    )
    
    models.append({f"asr": forward_asr, "synthesis": forward_synt, "dim": dim})

Loading 'exp/chain/e2e_tdnnf_wav2vec/final.pt'


CRITICAL:root:pkwrap: -- Failed to import kaldi you better not be in training mode (no backward possible) --


Loading 'exp/chain/e2e_tdnnf_wav2vec_lds_vq_48/final.pt'
Loading 'exp/chain/e2e_tdnnf/final.pt'
Loading 'exp/hifigan/g_00102000'
Loading 'exp/chain/e2e_tdnnf_vq_16/final.pt'
Loading 'exp/hifigan_vq_16_finetuned/g_00075000'
Loading 'exp/chain/e2e_tdnnf_vq_32/final.pt'
Loading 'exp/hifigan_vq_32_finetuned/g_00075000'
Loading 'exp/chain/e2e_tdnnf_vq_48/final.pt'
Loading 'exp/hifigan_vq_48_finetuned/g_00075000'
Loading 'exp/chain/e2e_tdnnf_vq_64/final.pt'
Loading 'exp/hifigan_vq_64_finetuned/g_00075000'
Loading 'exp/chain/e2e_tdnnf_vq_128/final.pt'
Loading 'exp/hifigan_vq_128_finetuned/g_00075000'
Loading 'exp/chain/e2e_tdnnf_vq_256/final.pt'
Loading 'exp/hifigan_vq_256_finetuned/g_00075000'
Loading 'exp/chain/e2e_tdnnf_vq_512/final.pt'
Loading 'exp/hifigan_vq_512_finetuned/g_00075000'
Loading 'exp/chain/e2e_tdnnf_vq_1024/final.pt'
Loading 'exp/hifigan_vq_1024_finetuned/g_best'


In [4]:
from ipywebrtc import AudioRecorder, CameraStream
import torch, torchaudio, os, tempfile, subprocess
from IPython.display import Audio, HTML
display(HTML("<style>div.output_scroll { height: 100em; }</style>"))

In [8]:
recordid="0010"

text_gt = subprocess.check_output(f"cat pkwrap/egs/librispeech/v1/data/vctk_test_trials_f_common/text | head -n{recordid} | tail -n1 | cut -d\  -f2-", shell=True).decode("UTF-8").strip()
print(text_gt)
camera = CameraStream(constraints={'audio': True,'video':False})
recorder = AudioRecorder(stream=camera)
display(recorder)

PEOPLE LOOK BUT NO ONE EVER FINDS IT


AudioRecorder(audio=Audio(value=b'', format='webm'), stream=CameraStream(constraints={'audio': True, 'video': …

In [9]:
# Recorded Audio from previous cell

if len(recorder.audio.value) != 0:
    tmpf = tempfile.NamedTemporaryFile(suffix='.webm')
    tmpfwav = tempfile.NamedTemporaryFile(suffix='.wav').name
    tmpf.write(recorder.audio.value)
    webm = tmpf.name
    !ffmpeg -i $webm -ac 1 -f wav $tmpfwav -y -hide_banner -loglevel panic 
    tmpf.close()
    waveform, sr = torchaudio.load(tmpfwav)

else:
    # waveform, sr = torchaudio.load(f"pkwrap/egs/librispeech/v1/data/vctk_test/wav/p226/p226_{recordid}_mic2.wav")

#     waveform, sr = torchaudio.load(f"speech-resynthesis/data/LJSpeech-1.1/wavs_16khz/LJ001-0011.wav")
#     text_gt = "IT IS OF THE FIRST IMPORTANCE THAT THE LETTER USED SHOULD BE FINE IN FORM"

    waveform, sr = torchaudio.load(f"pkwrap/egs/librispeech/v1/corpora/LibriSpeech/test-other/2414/128292/2414-128292-0015.flac")
    text_gt = "WHEN THE DEVIL CASTETH HIS SKIN DOTH NOT HIS NAME ALSO FALL AWAY IT IS ALSO SKIN"

#     waveform, sr = torchaudio.load(f"pkwrap/egs/librispeech/v1/corpora/LibriSpeech/test-other/2414/128292/2414-128292-0018.flac")
#     text_gt = "THEN ONLY DID I HIT THE TRUTH"

    # waveform, sr = torchaudio.load(f"pkwrap/egs/librispeech/v1/corpora/LibriSpeech/test-other/1688/142285/1688-142285-0015.flac")
    # text_gt = "HOW TAINTED ASKED HER FATHER"

#     waveform, sr = torchaudio.load(f"pkwrap/egs/librispeech/v1/corpora/LibriSpeech/test-other/6938/70848/6938-70848-0012.flac")
#     text_gt = "WHEREUPON THE OLD EXECUTIVE COMMITTEE LEFT THE HALL"

    # waveform, sr = torchaudio.load(f"speech-resynthesis/data/LJSpeech-1.1/wavs_16khz/LJ001-0011.wav")
    # text_gt = "IT IS OF THE FIRST IMPORTANCE THAT THE LETTER USED SHOULD BE FINE IN FORM"

#     waveform, sr = torchaudio.load(f"pkwrap/egs/librispeech/v1/corpora/LibriSpeech/dev-clean/1272/128104/1272-128104-0000.flac")
#     text_gt = "MISTER QUILTER IS THE APOSTLE OF THE MIDDLE CLASSES AND WE ARE GLAD TO WELCOME HIS GOSPEL"


waveform = torchaudio.transforms.Resample(orig_freq=sr,
                                                   new_freq=16000)(waveform)
waveform = waveform.to(demo.device)
Audio(data=waveform.cpu(), rate=16000)

In [10]:
import pkwrap
f0 = pkwrap.hifigan.f0.get_f0(
    waveform.cpu(), f0_stats_file="pkwrap/egs/LJSpeech/exp/f0_stats.pth"
).to(demo.device)
    
for model in models:
    print("Running for codebook size: " + str(model["dim"]))
    out, net = model["asr"](waveform.to(demo.device).clone())
    text = demo.kaldi_asr_decode(out)

    audio = model["synthesis"](audio=waveform.to(demo.device).clone(), f0=f0)[0]
    _,_, repport = demo.computeWER(text, text_gt)
    print(text)
    display(HTML(repport, metadata=dict(isolated=True)))
    print(f"Synthesis from ASR bottleneck {str(model['dim'])}: ")
    display(Audio(data=audio, rate=16000))
    
print("Wav2vec2:")
out, net = forward_asr_wav2vec2(waveform.to(demo.device).clone())
text = demo.kaldi_asr_decode(out)
print(text)
_,_, repport = demo.computeWER(text, text_gt)
display(HTML(repport, metadata=dict(isolated=True)))

print("Wav2vec2 + vq:48:")
out, net = forward_asr_wav2vec2_vq_48(waveform.to(demo.device).clone())
text = demo.kaldi_asr_decode(out)
print(text)
_,_, repport = demo.computeWER(text, text_gt)
display(HTML(repport, metadata=dict(isolated=True)))

Running for codebook size: no_vq
AN IMPERTINENT BUT NO END TO FIND IT


Synthesis from ASR bottleneck no_vq: 


Running for codebook size: 16
DICK NECK BUT NO ONE THE FINE ARTS


Synthesis from ASR bottleneck 16: 


Running for codebook size: 32
DEEPER NICK BUT NO ONE TO FIND IT


...

Synthesis from ASR bottleneck 512: 


Running for codebook size: 1024
IT DEPENDED BUT NO ONE TO FIND IT


Synthesis from ASR bottleneck 1024: 


Wav2vec2:
PEOPLE LOOK BUT NO ONE EVER FIND IT


Wav2vec2 + vq:48:
PEOPLE LOOK BUT NO ONE EVER FIND IT
